# Manila Flood Risk Map V6

## Project Overview
This notebook analyzes flood risk in Manila using geospatial and meteorological data. 
It calculates a composite **Risk Score** based on Elevation, Precipitation, Flood Height, and Slope.

### Enhancements in V6:
- **Grid-Based Visualization**: Replaced Heatmap with a Grid Map to strictly prioritize **Risk Score Intensity** over point density.
- **Max-Risk Coloring**: Each grid cell is colored based on the *maximum* risk score within it, ensuring isolated high-risk points are clearly visible.
- **Step-by-Step Analysis**: Code is split into distinct cells for better readability.
- **Advanced Visualization**: Scatter plots of Risk Score vs Features.
- **Interactive Map**: Enhanced Folium map with a custom Legend.
- **HTML Report Generation**: Automatically creates a standalone `project_presentation.html`.

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap, MarkerCluster
from scipy.spatial import cKDTree
from pathlib import Path
import os
import seaborn as sns
import matplotlib.pyplot as plt
import base64
from io import BytesIO

# --- Configuration ---
DATA_PATH = Path(os.path.expanduser('~/Desktop/AEGISDataset.csv'))
SLOPE_RADIUS_DEG = 0.01
KM_PER_DEGREE = 111.0

# Set Plot Style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("Libraries loaded.")

## 1. Data Loading and Preprocessing

In [ ]:
def load_and_preprocess(file_path):
    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")
    
    df = pd.read_csv(file_path)
    
    # Numeric conversion
    cols = ['flood_heig', 'elevation', 'precipitat', 'lat', 'lon']
    for c in cols: df[c] = pd.to_numeric(df[c], errors='coerce')
    
    df = df.dropna(subset=cols).copy()
    
    # Normalization Helper
    def norm(s, invert=False):
        mn, mx = s.min(), s.max()
        if mn == mx: return pd.Series(0.5, index=s.index)
        res = (s - mn) / (mx - mn)
        return 1 - res if invert else res
    
    df['norm_flood'] = norm(df['flood_heig'])
    df['norm_elev'] = norm(df['elevation'], invert=True)
    df['norm_precip'] = norm(df['precipitat'])
    
    return df

df = load_and_preprocess(DATA_PATH)
print(f"Data Loaded. Shape: {df.shape}")
df.head()

## 2. Slope Calculation (Vectorized)

In [ ]:
def calc_slope(df, radius=0.01):
    coords = df[['lat', 'lon']].values
    elevs = df['elevation'].values
    tree = cKDTree(coords)
    indices = tree.query_ball_point(coords, r=radius)
    
    slopes = []
    dist_m = radius * 111000
    
    for idx_list in indices:
        if len(idx_list) < 4:
            slopes.append(0.0)
            continue
        local_elevs = elevs[idx_list]
        diff = local_elevs.max() - local_elevs.min()
        deg = np.degrees(np.arctan(diff / dist_m))
        slopes.append(deg)
        
    df['slope'] = slopes
    
    # Normalize Slope (Inverted: High slope = Low Risk)
    mn, mx = df['slope'].min(), df['slope'].max()
    df['norm_slope'] = 1 - ((df['slope'] - mn) / (mx - mn))
    return df

df = calc_slope(df, SLOPE_RADIUS_DEG)

# Slope Distribution Plot
plt.figure(figsize=(8, 4))
sns.histplot(df['slope'], bins=30, kde=True, color='orange')
plt.title('Slope Distribution (Degrees)')
plt.xlabel('Slope')
plt.savefig('slope_dist.png') # Save for HTML report
plt.show()

## 3. Risk Score Calculation
Weights are calculated as the inverse of the mean absolute correlation. This assigns higher weights to features that provide unique information (low correlation).

In [ ]:
features = ['norm_flood', 'norm_elev', 'norm_precip', 'norm_slope']
corr = df[features].corr()

# Plot Correlation Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(corr, annot=True, cmap='RdYlGn', center=0, vmin=-1, vmax=1)
plt.title('Feature Correlation Matrix')
plt.savefig('corr_matrix.png')
plt.show()

# Calculate Weights
mean_corr = corr.abs().mean()
weights = 1 / (mean_corr + 1e-6)
weights = weights / weights.sum()

print("Weights:")
print(weights)

# Calculate Risk Score
df['risk_score'] = 0
for f in features:
    df['risk_score'] += df[f] * weights[f]
    
print(f"Risk Score Stats:\n{df['risk_score'].describe()}")

## 4. Feature Analysis (Scatter Plots)
Visualizing how the Risk Score relates to individual features.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.scatterplot(data=df, x='elevation', y='risk_score', ax=axes[0], hue='risk_score', palette='coolwarm')
axes[0].set_title('Risk vs Elevation')

sns.scatterplot(data=df, x='slope', y='risk_score', ax=axes[1], hue='risk_score', palette='coolwarm')
axes[1].set_title('Risk vs Slope')

sns.scatterplot(data=df, x='precipitat', y='risk_score', ax=axes[2], hue='risk_score', palette='coolwarm')
axes[2].set_title('Risk vs Precipitation')

plt.tight_layout()
plt.savefig('risk_scatter.png')
plt.show()

## 5. Interactive Map Generation
Creating a Folium map with a Heatmap, Markers, and a custom Legend.

In [ ]:
center_lat, center_lon = df['lat'].mean(), df['lon'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles='CartoDB positron')

# --- Grid-Based Visualization (Fix for Density Bias) ---
# Problem: Standard heatmaps prioritize density. Dense medium-risk areas look 'hotter' than isolated high-risk points.
# Fix: We divide the map into a grid. For each grid cell, we calculate the MAXIMUM risk score of points inside it.
# We then color the cell based on this max score. This ensures isolated high-risk points turn their area Red.

# 1. Create Grid Bins
# Define grid size (approx 0.005 degrees ~ 500m)
grid_size = 0.005
df['lat_bin'] = (df['lat'] // grid_size) * grid_size
df['lon_bin'] = (df['lon'] // grid_size) * grid_size

# 2. Aggregate by Grid Cell (Max Risk Score)
grid_df = df.groupby(['lat_bin', 'lon_bin'])['risk_score'].max().reset_index()

# 3. Define Color Function
def get_color(score):
    if score < 0.2: return 'green'
    elif score < 0.4: return 'yellow'
    elif score < 0.6: return 'orange'
    elif score < 0.8: return 'red'
    else: return 'darkred'

# 4. Add Grid Cells to Map
for _, row in grid_df.iterrows():
    folium.Rectangle(
        bounds=[[row['lat_bin'], row['lon_bin']], [row['lat_bin'] + grid_size, row['lon_bin'] + grid_size]],
        color=None,
        fill=True,
        fill_color=get_color(row['risk_score']),
        fill_opacity=0.6,
        popup=f"Max Risk: {row['risk_score']:.2f}"
    ).add_to(m)

# High Risk Markers (Keep these for specific point identification)
high_risk = df[df['risk_score'] > 0.8] # Explicit threshold for Danger Zones
cluster = MarkerCluster(name="High Risk Points").add_to(m)

for _, row in high_risk.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color='darkred',
        fill=True,
        fill_color='darkred',
        popup=f"Risk: {row['risk_score']:.2f}<br>Elev: {row['elevation']}m"
    ).add_to(cluster)

# Custom Legend HTML
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 110px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity:0.9; padding: 10px;">
     <b>Max Risk Level</b><br>
     <i style="background:darkred; width:10px; height:10px; display:inline-block;"></i> Very High (>0.8)<br>
     <i style="background:red; width:10px; height:10px; display:inline-block;"></i> High (0.6-0.8)<br>
     <i style="background:orange; width:10px; height:10px; display:inline-block;"></i> Medium (0.4-0.6)<br>
     <i style="background:yellow; width:10px; height:10px; display:inline-block;"></i> Low (0.2-0.4)<br>
     <i style="background:green; width:10px; height:10px; display:inline-block;"></i> Very Low (<0.2)
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

# Save Map
map_path = 'Manila_Flood_Risk_Map_V6.html'
m.save(map_path)
print(f"Map saved to {map_path}")
m

## 6. Generate HTML Presentation
This cell compiles the plots and map into a single HTML dashboard.

In [ ]:
def image_to_base64(img_path):
    with open(img_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# Encode images
slope_img = image_to_base64('slope_dist.png')
corr_img = image_to_base64('corr_matrix.png')
scatter_img = image_to_base64('risk_scatter.png')

# Read Map HTML (we'll embed it as an iframe source or directly)
# For simplicity in a single file, we can use an iframe pointing to the file if local, 
# or try to embed the map html content. Embedding complex folium maps directly into another HTML 
# can sometimes break scripts. Using an iframe is safer for the layout.

html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Manila Flood Risk Project</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 0; padding: 20px; background-color: #f4f4f9; }}
        h1 {{ color: #2c3e50; text-align: center; }}
        .container {{ max-width: 1200px; margin: auto; background: white; padding: 20px; box-shadow: 0 0 10px rgba(0,0,0,0.1); }}
        .section {{ margin-bottom: 40px; }}
        .plot-grid {{ display: flex; flex-wrap: wrap; justify-content: center; gap: 20px; }}
        .plot-card {{ flex: 1; min-width: 300px; text-align: center; }}
        img {{ max-width: 100%; height: auto; border: 1px solid #ddd; }}
        iframe {{ width: 100%; height: 600px; border: none; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>Manila Flood Risk Analysis</h1>
        
        <div class="section">
            <h2>Project Overview</h2>
            <p>This dashboard presents the analysis of flood risks in Manila. The risk score is calculated using a weighted model considering elevation, slope, precipitation, and historical flood height.</p>
        </div>

        <div class="section">
            <h2>Data Analysis</h2>
            <div class="plot-grid">
                <div class="plot-card">
                    <h3>Slope Distribution</h3>
                    <img src="data:image/png;base64,{slope_img}">
                </div>
                <div class="plot-card">
                    <h3>Feature Correlation</h3>
                    <img src="data:image/png;base64,{corr_img}">
                </div>
            </div>
            <div class="plot-card" style="margin-top: 20px;">
                <h3>Risk Score vs Features</h3>
                <img src="data:image/png;base64,{scatter_img}">
            </div>
        </div>

        <div class="section">
            <h2>Interactive Flood Risk Map</h2>
            <p>Explore the map below. Red areas indicate high risk. Markers show specific high-risk points.</p>
            <iframe src="Manila_Flood_Risk_Map_V4.html"></iframe>
        </div>
    </div>
</body>
</html>
"""

with open('project_presentation.html', 'w') as f:
    f.write(html_content)

print("HTML Presentation generated: project_presentation.html")